# M Serve Example
This Jupyter notebook runs M Serve on Colab.

## Install Ollama

Before we get started with Mellea, we download, install and serve ollama. We define set_css to wrap Colab output.

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh > /dev/null
!nohup ollama serve >/dev/null 2>&1 &

from IPython.display import HTML, display


def set_css():
    display(HTML("\n<style>\n pre{\n white-space: pre-wrap;\n}\n</style>\n"))


get_ipython().events.register("pre_run_cell", set_css)

## Install Mellea
We run `uv pip install mellea` to install Mellea.

In [ ]:
!uv pip install mellea -q

## Import Mellea and Start a Session
We initialize a backend and specify the context.

In [ ]:
from typing import Any

import mellea
from cli.serve.models import ChatMessage
from mellea.stdlib.context import ChatContext
from mellea.core import ModelOutputThunk, Requirement, SamplingResult
from mellea.stdlib.requirements import simple_validate
from mellea.stdlib.sampling import RejectionSamplingStrategy

session = mellea.start_session(ctx=ChatContext())


def validate_hi_bob(email: str) -> bool:
    return email.startswith("Hi Bob!")


def validate_email_len(email: str) -> bool:
    return len(email) < 50


def serve(
    input: list[ChatMessage],
    requirements: list[str] | None = None,
    model_options: None | dict = None,
) -> ModelOutputThunk | SamplingResult:
    """Takes a prompt as input and runs it through an M program."""
    requirements = requirements if requirements else []
    message = input[-1].content
    reqs = [
        Requirement(
            "Keep this under 50 words",
            validation_fn=simple_validate(validate_email_len),
        ),
        Requirement(
            "Add a 'Hi Bob!' at the top of the output",
            validation_fn=simple_validate(validate_hi_bob),
        ),
        *requirements,
    ]

    result = session.instruct(
        description=message,  # type: ignore
        requirements=reqs,  # type: ignore
        strategy=RejectionSamplingStrategy(loop_budget=3),
        model_options=model_options,
    )
    return result